<h1>
<center>
Dataquest Guided Project 20:
Practice Optimizing Dataframes and Processing in Chunks
</center>
</h1>

## Introduction

This is part of the Dataquest program.

- part of paths **Data Engineer**
    - Step 2: **Handling Large Data Sets in Python**
        - Course 1 :  **Processing Large Data sets in Pandas **
            - Optimizing Dataframe Memory Footprint
            - Processing Dataframes in Chunks
            - Augmenting Pandas with SQLite
       
As this is a guided project, we are following and deepening the steps suggested by Dataquest. In this project, we will practise working with chunked dataframes and optimizing a dataframe's memory usage.

## Use case : Lending Club

We will be working with financial lending data from [Lending Club](https://www.lendingclub.com), a marketplace for personal loans that matches borrowers with investors. 

The Lending Club's website lists approved loans. Qualified investors can view the borrower's credit score, the purpose of the loan, and other details in the loan applications. Once a lender is ready to back a loan it selects the amount of money it wants to fund. When the loan amount the borrower requested is fully funded, the borrower receives the money, minus the origination fee that Lending Club charges.

We will be working with a dataset of loans approved from 2007-2011, which we can download from the [Lending Club's website](https://www.lendingclub.com/info/download-data.action). If we read the entire data set, it will consume about 67 megabytes of memory. Let's imagine that we can only have 10 megabytes of memory available throughout this project.

## Load the data set

Let's first read in the first five lines for the file.

In [1]:
import pandas as pd
first_five = pd.read_csv('loans_2007.csv', nrows=5)
first_five

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


Let's read in the first 1000 rows from the data set, and calculate the total memory usage for these rows.

In [2]:
thousand_chunk = pd.read_csv('loans_2007.csv', nrows=1000)
thousand_chunk.memory_usage(deep=True).sum()/(1024*1024)

1.5502090454101562

## Exploring the Data in Chunks

Let's familiarize ourselves with the columns to see which ones we can optimize. 

### For each chunk, how many columns have a numeric type? How many have a string type?

In [3]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
i=0
for chunk in chunk_iter:
    print('Chunk #', i)
    print(chunk.dtypes.value_counts())
    i += 1

Chunk # 0
float64    30
object     21
int64       1
dtype: int64
Chunk # 1
float64    30
object     21
int64       1
dtype: int64
Chunk # 2
float64    30
object     21
int64       1
dtype: int64
Chunk # 3
float64    30
object     21
int64       1
dtype: int64
Chunk # 4
float64    30
object     21
int64       1
dtype: int64
Chunk # 5
float64    30
object     21
int64       1
dtype: int64
Chunk # 6
float64    30
object     21
int64       1
dtype: int64
Chunk # 7
float64    30
object     21
int64       1
dtype: int64
Chunk # 8
float64    30
object     21
int64       1
dtype: int64
Chunk # 9
float64    30
object     21
int64       1
dtype: int64
Chunk # 10
float64    30
object     21
int64       1
dtype: int64
Chunk # 11
float64    30
object     21
int64       1
dtype: int64
Chunk # 12
float64    30
object     21
int64       1
dtype: int64
Chunk # 13
float64    30
object     22
dtype: int64
Chunk # 14
float64    30
object     22
dtype: int64


There are 31 numeric columns and 21 string columms by default.Let's explore what is happening in the 2 lasts chunks where the number of string columns is 22.

In [4]:
# Are string columns consistent across chunks?
obj_cols = []
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)

i=0
for chunk in chunk_iter:
    chunk_obj_cols = chunk.select_dtypes(include=['object']).columns.tolist()
    if len(obj_cols) > 0:
        is_same = obj_cols == chunk_obj_cols
        if not is_same:
            print ("--- Chunk #", i, "----")
            print("overall obj cols:", obj_cols, "\n")
            print("chunk obj cols:", chunk_obj_cols, "\n")    
    else:
        obj_cols = chunk_obj_cols
    i += 1

--- Chunk # 13 ----
overall obj cols: ['term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type'] 

chunk obj cols: ['id', 'term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type'] 

--- Chunk # 14 ----
overall obj cols: ['term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'las

It seems like one column in particular (the id column) is being cast to int64 in the last 2 chunks but not in the earlier chunks. Since the id column won't be useful for analysis, visualization, or predictive modelling let's ignore this column.

### For Each Chunk how many unique values are there in each string column? What is the percent of unique values?

Let's build dictionaries to answer accurately to this question for each chunk.

In [5]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
str_cols_vc = dict()
unique_percent = dict()
for chunk in chunk_iter:
    str_cols = chunk.select_dtypes(include=['object'])
    for col in str_cols.columns:
        current_col_vc = str_cols[col].nunique()
        total_len = len(str_cols[col])
        if col in str_cols_vc:
            str_cols_vc[col].append(current_col_vc)
            unique_percent[col].append(1 - current_col_vc/total_len)
        else:
            str_cols_vc[col] = [current_col_vc]
            unique_percent[col] = [1 - current_col_vc/total_len]

In [6]:
str_cols_vc

{'addr_state': [43, 45, 43, 43, 43, 43, 43, 43, 43, 42, 41, 42, 47, 50, 42],
 'application_type': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'earliest_cr_line': [366,
  399,
  401,
  383,
  391,
  383,
  385,
  387,
  374,
  367,
  363,
  373,
  366,
  381,
  218],
 'emp_length': [11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11],
 'emp_title': [2653,
  2588,
  2592,
  2598,
  2590,
  2648,
  2600,
  2497,
  2660,
  2654,
  2644,
  2554,
  2598,
  2605,
  477],
 'grade': [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7],
 'home_ownership': [3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 4, 4, 4, 5, 4],
 'id': [3000, 538],
 'initial_list_status': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'int_rate': [36, 36, 66, 37, 73, 68, 45, 65, 68, 50, 60, 65, 107, 262, 86],
 'issue_d': [2, 3, 3, 3, 3, 3, 4, 3, 4, 4, 5, 7, 17, 43, 9],
 'last_credit_pull_d': [55,
  57,
  59,
  59,
  61,
  62,
  65,
  67,
  70,
  72,
  77,
  83,
  97,
  105,
  89],
 'last_pymnt_d': [54, 56, 57, 59, 60, 62, 64, 65, 66,

In [7]:
unique_percent

{'addr_state': [0.9856666666666667,
  0.985,
  0.9856666666666667,
  0.9856666666666667,
  0.9856666666666667,
  0.9856666666666667,
  0.9856666666666667,
  0.9856666666666667,
  0.9856666666666667,
  0.986,
  0.9863333333333333,
  0.986,
  0.9843333333333333,
  0.9833333333333333,
  0.9219330855018587],
 'application_type': [0.9996666666666667,
  0.9996666666666667,
  0.9996666666666667,
  0.9996666666666667,
  0.9996666666666667,
  0.9996666666666667,
  0.9996666666666667,
  0.9996666666666667,
  0.9996666666666667,
  0.9996666666666667,
  0.9996666666666667,
  0.9996666666666667,
  0.9996666666666667,
  0.9996666666666667,
  0.9981412639405205],
 'earliest_cr_line': [0.878,
  0.867,
  0.8663333333333334,
  0.8723333333333333,
  0.8696666666666667,
  0.8723333333333333,
  0.8716666666666667,
  0.871,
  0.8753333333333333,
  0.8776666666666667,
  0.879,
  0.8756666666666667,
  0.878,
  0.873,
  0.5947955390334573],
 'emp_length': [0.9963333333333333,
  0.9963333333333333,
  0.99633333

### For each chunk, which  float columns have no missing values and could be candidates for conversion to the integer type?

Let's build dictionaries to answer accurately to this question for each chunk.

In [8]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
float_cols_vc = dict()
unique_percent = dict()
is_integer = dict()
for chunk in chunk_iter:
    float_cols = chunk.select_dtypes(include=['float'])
    for col in float_cols.columns:
        current_col_vc = float_cols[col].isnull().sum()
        total_len = len(float_cols[col])
        integer = float_cols[col].apply(float.is_integer).all()
        if col in float_cols_vc:
            float_cols_vc[col].append(current_col_vc)
            unique_percent[col].append(current_col_vc/total_len)
            is_integer[col].append(integer)
        else:
            float_cols_vc[col] = [current_col_vc]
            unique_percent[col] = [current_col_vc/total_len]
            is_integer[col] = [integer]

In [9]:
float_cols_vc

{'acc_now_delinq': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 31],
 'annual_inc': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 6],
 'chargeoff_within_12_mths': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 57, 91],
 'collection_recovery_fee': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2],
 'collections_12_mths_ex_med': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 57, 91],
 'delinq_2yrs': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 31],
 'delinq_amnt': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 31],
 'dti': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2],
 'funded_amnt': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2],
 'funded_amnt_inv': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2],
 'inq_last_6mths': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 31],
 'installment': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2],
 'last_pymnt_amnt': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2],
 'loan_amnt': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2],
 'member_id': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2],
 'open

In [10]:
unique_percent

{'acc_now_delinq': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0003333333333333333,
  0.05762081784386617],
 'annual_inc': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0003333333333333333,
  0.011152416356877323],
 'chargeoff_within_12_mths': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.019,
  0.1691449814126394],
 'collection_recovery_fee': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0003333333333333333,
  0.0037174721189591076],
 'collections_12_mths_ex_med': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.019,
  0.1691449814126394],
 'delinq_2yrs': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0003333333333333333,
  0.05762081784386617],
 'delinq_amnt': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  

In [11]:
is_integer

{'acc_now_delinq': [True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  False],
 'annual_inc': [False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False],
 'chargeoff_within_12_mths': [True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  False],
 'collection_recovery_fee': [False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False],
 'collections_12_mths_ex_med': [True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  False],
 'delinq_2yrs': [True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  False,
  False],
 'delinq_amnt': [True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
  True,
 

### Calculate the total memory usage across all of the chunks

In [12]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
for chunk in chunk_iter:
    print(chunk.memory_usage(deep=True).sum()/(1024*1024))

4.649013519287109
4.6447601318359375
4.646517753601074
4.647870063781738
4.6440629959106445
4.6459455490112305
4.644536972045898
4.646905899047852
4.645031929016113
4.645082473754883
4.657794952392578
4.6566619873046875
4.663469314575195
4.896910667419434
0.8808088302612305


How many rows are there? 

In [13]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000)
total_rows = 0
for chunk in chunk_iter:
    total_rows += len(chunk)
print(total_rows)

42538


## Optimizing String Columns

We can achieve the greatest memory improvements by converting the string columns to a numeric type. Let's convert some colums with the less unique values to the category type, and the columms that contain numeric values to the float type.

Let's focus on columns that would actually be useful for analysis and modelling.

In [14]:
useful_obj_cols = ['term', 'sub_grade', 'emp_title', 'home_ownership', 'verification_status', 'issue_d', 'purpose', 'earliest_cr_line', 'revol_util', 'last_pymnt_d', 'last_credit_pull_d']

The columns we'll convert to category are the following :

In [ ]:
convert_col_dtypes = {
    "sub_grade": "category", "home_ownership": "category", 
    "verification_status": "category", "purpose": "category"
}

## Optimizing Numeric columns

Let's convert issue_d, earliest_cr_line, last_pymnt_d, and last_credit_pull_d to datetime and convert term and revol_util to numerical by data cleaning.

In [ ]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=3000, dtype=convert_col_dtypes, parse_dates=["issue_d", "earliest_cr_line", "last_pymnt_d", "last_credit_pull_d"])
mv_counts = {}
for chunk in chunk_iter:
    term_cleaned = chunk['term'].str.lstrip(" ").str.rstrip(" months")
    revol_cleaned = chunk['revol_util'].str.rstrip("%")
    chunk['term'] = pd.to_numeric(term_cleaned)
    chunk['revol_util'] = pd.to_numeric(revol_cleaned)
    print(chunk.memory_usage(deep=True).sum()/(1024*1024))

2.943678855895996
2.939072608947754
2.94156551361084
2.9422073364257812
2.9393367767333984
2.9405736923217773
2.9395084381103516
2.941800117492676
2.9403858184814453
2.941774368286133
2.954832077026367
2.955653190612793
2.963329315185547


The total memory footprint has been improved thanks to the changes we made.